Name : Maurya Vijayaramchandran 

Project : Building a convolutional Neural net on CIFAR10 dataset 

Import the libraries

In [ ]:
import os 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

Load the dataset and split it into test and train sets 
1. The test and train sets are further split into 
a. Features X_train , X_test
b. Labels y_train, y_test

In [ ]:
(x_train, y_train), (x_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 19s 0us/step


Analyze the shape of the dataset 

In [ ]:
x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0

Creating the model

In [ ]:
"""
Create a CNN model using Keras API of TensorFlow.

Parameters:
    None

Returns:
    keras.Sequential: A sequential model object.

Architecture:
    - Input layer: accepts 32x32 RGB images
    - Conv2D layer with 32 filters of size (3,3), ReLU activation, and valid padding
    - MaxPooling2D layer with pool size (2,2)
    - Conv2D layer with 64 filters of size 3x3, ReLU activation, and default padding
    - MaxPooling2D layer with default pool size (2,2)
    - Conv2D layer with 128 filters of size 3x3, ReLU activation, and default padding
    - Flatten layer to convert 2D features to 1D features
    - Dense layer with 64 neurons and ReLU activation
    - Dense output layer with 10 neurons for classification

Note:
    - The model is compiled on Colab with GPU.
    - The output layer has no activation function, so it returns logits.
"""

model = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)),
     layers.Conv2D(32,(3,3), padding = "valid", activation = "relu"),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64,3, activation = "relu"),
     layers.MaxPooling2D(),
     layers.Conv2D(128,3,activation = "relu"),
     layers.Flatten(),
     layers.Dense(64,activation = "relu"),
     layers.Dense(10)

    ]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

compiling the model

In [ ]:
"""
Compilation block

Parameters:
    loss: str or callable. The loss function to use. If a string, the name of the built-in Keras loss function. If a callable, the custom loss function to use. Default is SparseCategoricalCrossentropy from logits.
    optimizer: str, callable, or tf.keras.optimizers.Optimizer object. The optimizer to use during training. If a string, the name of the built-in Keras optimizer. If a callable, the custom optimizer to use. Default is Adam with learning rate of 0.01.
    metrics: list of str. The evaluation metrics to use during training and testing. If not provided, the default metric for each model will be used.

Returns:
    None

Note:
    - This method must be called before training the model.
    - It is possible to pass custom metrics as a function or a list of functions.
"""

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits= True),
    optimizer = keras.optimizers.Adam(learning_rate=0.01),
    metrics = ["accuracy"],

)

modifying the model as a functional model

It is overfitting, let us add regularization

In [ ]:
def my_model():
"""
Define a custom Keras model.

Returns:
    A Keras model object.

Note:
    - This model consists of multiple convolutional layers, batch normalization, activation functions, max pooling, and dense layers.
    - Regularization techniques such as L2 regularization and dropout are also used.
    - This function returns the defined Keras model object.
"""

  inputs = keras.Input(shape= (32,32,3))
  x = layers.Conv2D(32,3, padding = "same",
                    kernel_regularizer = regularizers.l2(0.01)
                    )(inputs)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64,5,padding = "same",
                    kernel_regularizer = regularizers.l2(0.01))(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128,3,padding = "same",
                    kernel_regularizer = regularizers.l2(0.01))(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation="relu",kernel_regularizer = regularizers.l2(0.01))(x)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs = inputs, outputs = outputs)
  return model
model = my_model()
print(model.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_13 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 tf.nn.relu_13 (TFOpLambda)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 16, 16, 64)        5126

Compiling the functional model

In [ ]:
"""
Compile block

Parameters:
    loss: str or callable. The loss function to use. If a string, the name of the built-in Keras loss function. If a callable, the custom loss function to use. Default is SparseCategoricalCrossentropy from logits.
    optimizer: str, callable, or tf.keras.optimizers.Optimizer object. The optimizer to use during training. If a string, the name of the built-in Keras optimizer. If a callable, the custom optimizer to use. Default is Adam with learning rate of 0.01.
    metrics: list of str. The evaluation metrics to use during training and testing. If not provided, the default metric for each model will be used.

Returns:
    None

Note:
    - This method must be called before training the model.
    - It is possible to pass custom metrics as a function or a list of functions.
"""

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits= True),
    optimizer = keras.optimizers.Adam(learning_rate=0.01),
    metrics = ["accuracy"],

)

Training and testing the model.

In [ ]:
"""
Train and evaluate a Keras model.

Parameters:
    x_train: array-like. The training data.
    y_train: array-like. The training labels.
    x_test: array-like. The testing data.
    y_test: array-like. The testing labels.
    batch_size: int. The number of samples per gradient update. Default is 64.
    epochs: int. The number of times to iterate over the entire training dataset. Default is 150.
    verbose: int. Verbosity mode (0 = silent, 1 = progress bar, 2 = one line per epoch). Default is 2.

Returns:
    A tuple of test loss and test accuracy values.

Note:
    - This method fits the Keras model on the training dataset and evaluates the trained model on the testing dataset.
    - It is recommended to use a validation dataset to monitor the performance of the model during training.
"""

model.fit(x_train, y_train, batch_size=64, epochs= 150, verbose = 2)
model.evaluate(x_test,y_test, batch_size=32, verbose = 2)